In [1]:
import pandas as pd
from datetime import datetime, timedelta
from binance import Client
import sched, time
from ta import add_all_ta_features
from ta.utils import dropna
import warnings
warnings.filterwarnings('ignore')
import pickle
import requests

In [2]:
def telegram_bot_sendtext(bot_message):
    try:
        bot_token = '5275451140:AAH9f6aVjMd0K6opDKn-qet6LA8dr6L52_8'
        bot_chatID = '-1001588160937'
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
        response = requests.get(send_text)
        return response.json()
    except(e):
        print(e)
        print("Failed to send telegram message")

In [3]:
tick_interval = "1m"
sleep_time = 60
samples = 100
client = Client()
symbol = 'MATICUSDT'
last_prediction = -1

In [4]:
with open('RandomForest_depth_4.pickle', 'rb') as f:
    clf = pickle.load(f)

In [5]:
df = pd.DataFrame({'openTime': pd.Series(dtype='int64'),
                   'open': pd.Series(dtype='float64'),
                   'high': pd.Series(dtype='float64'),
                   'low': pd.Series(dtype='float64'),
                   'close': pd.Series(dtype='float64'),
                   'volume': pd.Series(dtype='float64'),
                   'closeTime': pd.Series(dtype='int64')})

In [6]:
telegram_bot_sendtext('Start: ' + datetime.now().strftime("%Y/%m/%d, %H:%M:%S"))
s = sched.scheduler(time.time, time.sleep)

def prediction_loop(): 
    global df
    global clf
    global last_prediction
    
    now = datetime.now()
#     print('Time: ' + str(now))
    startTime = int((now - timedelta(minutes=5)).timestamp() * 1000)
    
    if len(df) < 30:
        fallBack = int((datetime.now() - timedelta(minutes=35)).timestamp() * 1000)
        candles = client.get_klines(symbol=symbol, interval=tick_interval, startTime=fallBack, endTime=None, limit=50)
        if len(candles) > 30:
            candles = candles[-30:]
    else:
        candles = client.get_klines(symbol=symbol, interval=tick_interval, startTime=startTime, endTime=None, limit=10)
        if len(candles) > 0:
            candles = candles[-1:]
            
    if len(candles) > 0:
        for candle in candles:
            candle_df = pd.DataFrame({'openTime': pd.Series([int(candle[0])], dtype='int64'),
                           'open': pd.Series([float(candle[1])], dtype='float64'),
                           'high': pd.Series([float(candle[2])], dtype='float64'),
                           'low': pd.Series([float(candle[3])], dtype='float64'),
                           'close': pd.Series([float(candle[4])], dtype='float64'),
                           'volume': pd.Series([float(candle[5])], dtype='float64'),
                           'closeTime': pd.Series([int(candle[6])], dtype='int64')})
            while len(df) >= samples:
                df.drop([0], inplace=True)
            df = df.append(candle_df, ignore_index=True)
            
#     print('\nDATAFRAME: ')
#     print(df)
    
    if len(df) >= 30:
        df_ta = dropna(df)
        df_ta = add_all_ta_features(
            df_ta, open="open", high="high", low="low", close="close", volume="volume")
        df_ta = df_ta.fillna(0)

        df_ta['openTime'] = pd.to_datetime(df_ta['openTime'])
        df_ta['openTime'] = df_ta['openTime'].apply(lambda x: x.toordinal())
        df_ta['closeTime'] = pd.to_datetime(df_ta['closeTime'])
        df_ta['closeTime'] = df_ta['closeTime'].apply(lambda x: x.toordinal())

        predictions = clf.predict(df_ta)
#         print('Predictions: ' + str(predictions[-10:]))
#         print('Last prediction: ' + str(last_prediction))
        telegram_bot_sendtext('Signal: ' + str(int(predictions[-1])) + '; ' + str(now.strftime("%Y/%m/%d, %H:%M:%S")) +
                             '\nPrice: ' + str(df_ta.iloc[len(df_ta) - 1]['close']))
        
        if last_prediction == 0 and predictions[-1] == 1:
            telegram_bot_sendtext('BUY: ' + str(df_ta.iloc[len(df_ta) - 1]['close']))
#             print('BUY: ' + str(df_ta.iloc[len(df_ta) - 1]['close']))
            
        elif last_prediction == 1 and predictions[-1] == 0:
            telegram_bot_sendtext('SELL: ' + str(df_ta.iloc[len(df_ta) - 1]['close']))
#             print('SELL: ' + str(df_ta.iloc[len(df_ta) - 1]['close']))
            
    last_prediction = predictions[-1]
    
    s.enter(60, 1, prediction_loop)

s.enter(60, 1, prediction_loop)
s.run()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

ReadTimeout: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out. (read timeout=10)